In [3]:
import sys
sys.path.append("../") # Add the parent directory to the path
from utils.get_data import MultiModalAugmentation
from objective_augment import process_augment_config
import yaml

In [39]:
from torchvision.models.mobilenetv3 import mobilenet_v3_small
from torch import nn

mobilenet = mobilenet_v3_small(pretrained=True)
mobilenet.features[0][0] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False)
# mobilenet.features[1].block[0][0].weight

c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [38]:
import torch
import torchvision

resnet = torchvision.models.resnet18(num_classes=512)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet
# (list(resnet.children()))
# state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
# resnet.load_state_dict(state_dict)
# resnet.eval()

# # get resnet model
# resnet = torch.nn.Sequential(*(list(resnet.children()))[:-2])

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [37]:
import optuna
from optuna.distributions import FloatDistribution

trial = optuna.trial.FixedTrial({
    "local_views.random_affine.scale": [0.5, 0.9]
})

In [ ]:
config_path = '../configs/config_multimodal_dino.yaml'
config = yaml.safe_load(open(config_path))
best_augments = process_augment_config(None, config, is_hyperparameter_search=False) # IMPORTANT, set to FALSE
multimodal_augments = MultiModalAugmentation(augment_values=best_augments)
# print(f"{str(multimodal_augments)}")
param_info = config['optuna']['augmentations']['local_views']['random_resized_crop']['scale']
print(param_info)
print(trial.suggest_categorical(
        f"local_views.random_affine.scale",
        param_info["choices"]))


{'type': 'categorical', 'choices': [[0.3, 0.7], [0.4, 0.8], [0.5, 0.9]]}
[0.5, 0.9]


c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0.3, 0.7] which is of type list.
  warnings.warn(message)
c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0.4, 0.8] which is of type list.
  warnings.warn(message)
c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [0.5, 0.9] which is of type list.
  warnings.warn(message)


In [4]:
import torchaudio.transforms as audio_transforms
str(audio_transforms.TimeMasking(time_mask_param=15).mask_param)

'15'

In [5]:
from models.dino import MultiModalDINOLightning
import copy

model = MultiModalDINOLightning(data_dir='../data/avmnist')
initial_weights = copy.deepcopy(model.state_dict())

seeds = [42, 43, 44, 45, 46]
for seed in seeds:
    model.load_state_dict(copy.deepcopy(initial_weights))

In [1]:
class BaseModel:
    def __init__(self, **kwargs):
        self.dropout = kwargs.get("dropout", 0.5)
        self.hidden_size = kwargs.get("hidden_size", 128)

class MyModel(BaseModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        print("Dropout:", kwargs.get("dropout"))
        print("Hidden Size:", kwargs.get("hidden_size"))

model = MyModel(dropout=0.3, hidden_size=256)

Dropout: 0.3
Hidden Size: 256


In [7]:
import pytorch_lightning as pl

class LightningModuleTest(pl.LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        print("Hyperparameters:", self.hparams)

    def forward(self, x):
        return x
    
model = LightningModuleTest(dropout=0.3, hidden_size=256)
LightningModuleTest.__class__

Hyperparameters: "dropout":     0.3
"hidden_size": 256


type